# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [14]:
# Imports
import pandas as pd
import numpy as np
from scipy import stats

# Formats my Pandas DataFrames to be more readable
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css)) 

In [15]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


# Introductory Analysis
I created a contingency table in order to inspect differences between call backs for black-sounding names and white-sounding names. 

In [16]:
contingency_table = pd.crosstab(data.race, data.call, margins=True)
contingency_table.columns = ['No Call', 'Call Back', 'Total']
contingency_table['Race'] = ['Black', 'White', 'Total']
contingency_table = contingency_table.set_index('Race')
contingency_table['Percent Called Back'] = pd.DataFrame(round(100*contingency_table['Call Back']/contingency_table['Total'], 2))
contingency_table

,No Call,Call Back,Total,Percent Called Back
Race,,,,
Black,2278,157,2435,6.45
White,2200,235,2435,9.65
Total,4478,392,4870,8.05


One can see that there are potential differences between how frequently black-sounding names and white-sounding names received return calls. Whether this is statistically significant will be explored in this document. 

# Does the Central Limit Theorem apply?

See this website for a basic summary of the assumptions and conditions of the central limit theorem: http://archive.cnx.org/contents/ee652647-7d10-4c33-b439-9bdf5aa6ba3f@1/central-limit-theorem-assumptions-and-conditions. Each of these is met in this experiment: 

**Randomination Condition:** The racial category for each application used was chosen randomly within proper randominization protocol. 

**Independence Assumption:** This is subsumed within the randomization condition. 

**10% Condition:** The population here is indefinitely large (all potential applicants for all jobs), so the sample size is clearly less than 10%. 

**Sample Size Assumption:** This is an indefinitely large population, so this condition is met. 

Thus, we can assume the data is normally distributed. 

In [17]:
# For all hypothesis tests in this report, my alpha level will be 0.05. 
alpha = 0.05 

# Chi-Squared Test

I will conduct a chi-squared test to determine whether the difference between call backs between black-sounding names and white-sounding names is statistically significant. 

***Null Hypothesis:*** Call are backs are uniformly distributed over both black-sounding names and white-sounding names (aka, the racial association of the name and call backs are independent). 

***Alternative Hypothesis:*** Call backs are not uniformly distributed over both black-sounding names and white-sounding names (aka, the racial association of the name and call backs are dependent).

In [18]:
(chi, p, dof, expected) = stats.chi2_contingency(pd.crosstab(data.race, data.call))

print("chi = " + str(chi) + " and p = "+ str(p))

print(" ")

if(p < alpha): 
    print("I reject the null.")
else: 
    print("I fail to the reject the null.")

chi = 16.4490285842 and p = 4.99757838996e-05
 
I reject the null.


The p-value is really small. We thus cannot conclude that the call backs are dependent on the racial associations of the names. 

Given the lower rate of call backs for blacks, I can therefore reasonably conclude that most likely, blacks received call backs less frequently than whites. 

## Confidence Interval

I will also calculate the 95% confidence interval on the total number of people who received call backs to see whether the ratio of resumes with black-sounding names called back or resumes with white-sounding names called back were within the range. This will also tell us determine whether the difference in proportions is significant. 

In [19]:
# n is our sample size 
n = len(data)
sample_proportion = len(data[data.call == 1]) / n
# We will use the standard error to estimate the standard deviation of the population. 
standard_error = ( sample_proportion*(1 - sample_proportion) / n )**(0.5)
(low, high) = stats.norm.interval(1-alpha, loc=sample_proportion, scale = standard_error)
print( 'There is a ' + str ((1-alpha)*100) + '% that the actual number of people who received call backs on their resume in the population is between ' + str(round(low*100, 2)) + '% and ' + str(round(high*100, 2)) + '%.')

There is a 95.0% that the actual number of people who received call backs on their resume in the population is between 7.29% and 8.81%.


Neither the percent of white names called back (9.65%) nor the percent of black names called back (6.45%) fall within this 95% confidence interval. If the chances of each group receiving a call back were independent of their racial group, then we would expect them to fall within this range 95% of the time. The fact that they are very from the range further confirms the unlikeliness that call backs is independent of the racial categorization of the name. 

# Conclusion

We are left to conclude that racial bias based on whether a name sounds white or black is extremely likely when employers are considering who to call back. It is unclear whether the racial association of the name is the most important factor in call back success. Further study would reveal its explanatory significance relative to other features. But it certianly influences the result. This substantiates theories of racial bias in the job application system. 